# 패스트캠퍼스 바이트디그리 최종 프로젝트 과제 소개

## Load Dataset

In [ ]:
from google.colab import drive

import pandas as pd

In [ ]:
order = pd.read_csv('/content/sample_data/order.csv')
course = pd.read_csv('/content/sample_data/course.csv')
customer = pd.read_csv('/content/sample_data/customer.csv')
refund = pd.read_csv('/content/sample_data/refund.csv')

## Q1. Data Extract - SQL


- Q1-1. 4개의 테이블(order, customer, course, refund)을 하나의 테이블로 조인하는 쿼리를 작성해주세요.
- Q1-2. (Order) 2022년 1월 일자별 가장 빠르게 신청된 강의(created_at)와 가장 나중에 신청된 강의를 볼 수 있는 쿼리를 작성해 주세요. (First_Value, Last_value, partition by)
- Q1-3. Order 테이블의 list_price를 일자별(created_at)로 합계한 후, 1일 전 과 1일 후의 매출을 column으로 만들어 주세요. (lead, lag)

## Q2. Data Analysis - Python 2022년 1월 기준

- Q2-1 (Order) 2022년 1월 중 가장 매출이 높았던 3일과 낮았던 3일은 언제인가요?
(status = COMPLETED 기준)
- Q2-2 (Order) 그리고 그 때의 매출액(list_price)과, 주문 수, 신청자 수, 강의 수는 각각 얼마인가요?
- Q2-3 (Order) 각 강의 별 할인율은 어떻게 될까요?
- Q2-4 (Customer) 회원들은 주로 몇시에 회원 가입을 했나요?
- Q2-5 (Refund) 환불이 가장 많은 강의는 어떤 강의였나요?
- Q2-6 (Customer) 강의를 가장 많이 신청한 고객과 그 금액은 얼마인가요?
- Q2-7 (Order) 주말과 주중의 평균 강의 신청율은 어떻게 다를까요?
- Q2-8 (Course) Keyword 열에서 가장 많이 반복되는 단어는 어떤 단어인가요?
- Q2-9 (Order) 강의 가격(list_price)을 5만원 단위의 도수분포표로 나타내 주세요.
- Q2-10 (Order) 신청이 완료된 강의(complated)와 진행중인 강의(pending), 그리고 취소된 강의(cancelled) 간의 금액(list_price)과 유저의 비중은 각각 어떻게 될까요?

## Q3. Data Visualization - Excel, Tableau, Power BI, python(matplolib, seaborn)
Q2에 대한 답을 하면서 적절하다고 생각하는 차트를 만들어주세요

## Q4. Analytics Report
데이터를 보면서 데이터 분석가로서 찾아낸 인사이트를 공유해주세요. 데이터 분석가마다 보는 관점이 다르고 나타내는 방법도 다양합니다. 데이터 분석을 잘 모르는 비즈니스 유저에게 본인의 분석 결과를 설명한다고 가정하고 어떤 제안을 할 수 있는지 생각해 보면 좋을 것 같습니다.